In [232]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [233]:
#!pip install statsmodels

In [234]:
#importing libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
# import lazypredict

from plotly.subplots import make_subplots
import plotly.graph_objects as go
import warnings
warnings.filterwarnings("ignore")

import statsmodels.tsa.stattools as st
from statsmodels.tsa.arima_model import ARMA
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_absolute_error
from statsmodels.tsa.arima.model import ARIMA

In [235]:
df = pd.read_csv('Data/GCB2022v27_MtCO2_flat.csv')
df

,Country,ISO 3166-1 alpha-3,Year,Total,Coal,Oil,Gas,Cement,Flaring,Other,Per Capita
0,Afghanistan,AFG,1750,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,AFG,1751,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,AFG,1752,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,AFG,1753,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,AFG,1754,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
63099,Global,WLD,2017,36096.739276,14506.973805,12242.627935,7144.928128,1507.923185,391.992176,302.294047,4.749682
63100,Global,WLD,2018,36826.506600,14746.830688,12266.016285,7529.846784,1569.218392,412.115746,302.478706,4.792753
63101,Global,WLD,2019,37082.558969,14725.978025,12345.653374,7647.528220,1617.506786,439.253991,306.638573,4.775633
63102,Global,WLD,2020,35264.085734,14174.564010,11191.808551,7556.290283,1637.537532,407.583673,296.301685,4.497423


In [236]:
#dropping years less than 1850
df.drop(df[df['Year'] < 1850].index, inplace = True)

In [237]:
df = df.reset_index()

In [238]:
df.columns

Index(['index', 'Country', 'ISO 3166-1 alpha-3', 'Year', 'Total', 'Coal',
       'Oil', 'Gas', 'Cement', 'Flaring', 'Other', 'Per Capita'],
      dtype='object')

In [239]:
## Filling nans with zero for the inpuut features
df.Coal = df['Coal'].fillna(0, inplace = False)
df.Oil = df['Oil'].fillna(0, inplace = False)
df.Gas = df['Gas'].fillna(0, inplace = False)
df.Cement = df['Cement'].fillna(0, inplace = False)
df.Flaring = df['Flaring'].fillna(0, inplace = False)
df.Other = df['Other'].fillna(0, inplace = False)
df['Per Capita'] = df['Per Capita'].fillna(0, inplace = False)

In [240]:
# Checking the count of null values
df.isna().sum()

index                    0
Country                  0
ISO 3166-1 alpha-3    1032
Year                     0
Total                  100
Coal                     0
Oil                      0
Gas                      0
Cement                   0
Flaring                  0
Other                    0
Per Capita               0
dtype: int64

In [241]:
df.reset_index()
df

,index,Country,ISO 3166-1 alpha-3,Year,Total,Coal,Oil,Gas,Cement,Flaring,Other,Per Capita
0,100,Afghanistan,AFG,1850,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,101,Afghanistan,AFG,1851,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,102,Afghanistan,AFG,1852,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,103,Afghanistan,AFG,1853,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,104,Afghanistan,AFG,1854,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
39899,63099,Global,WLD,2017,36096.739276,14506.973805,12242.627935,7144.928128,1507.923185,391.992176,302.294047,4.749682
39900,63100,Global,WLD,2018,36826.506600,14746.830688,12266.016285,7529.846784,1569.218392,412.115746,302.478706,4.792753
39901,63101,Global,WLD,2019,37082.558969,14725.978025,12345.653374,7647.528220,1617.506786,439.253991,306.638573,4.775633
39902,63102,Global,WLD,2020,35264.085734,14174.564010,11191.808551,7556.290283,1637.537532,407.583673,296.301685,4.497423


In [242]:
df['Country'].unique()

array(['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola',
       'Anguilla', 'Antarctica', 'Antigua and Barbuda', 'Argentina',
       'Armenia', 'Aruba', 'Australia', 'Austria', 'Azerbaijan',
       'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus',
       'Belgium', 'Belize', 'Benin', 'Bermuda', 'Bhutan',
       'Bonaire, Saint Eustatius and Saba', 'Bosnia and Herzegovina',
       'Botswana', 'Brazil', 'British Virgin Islands',
       'Brunei Darussalam', 'Bulgaria', 'Burkina Faso', 'Burundi',
       'Cambodia', 'Canada', 'Cape Verde', 'Central African Republic',
       'Chad', 'Chile', 'China', 'Christmas Island', 'Colombia',
       'Comoros', 'Congo', 'Cook Islands', 'Costa Rica', "Côte d'Ivoire",
       'Croatia', 'Cuba', 'Curaçao', 'Cyprus', 'Czech Republic',
       'North Korea', 'Democratic Republic of the Congo', 'Denmark',
       'Djibouti', 'Dominica', 'Dominican Republic', 'Ecuador', 'Egypt',
       'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Estonia',
   

List of countries: USA, Japan, Germany, China, India, Brazil, Senegal, Cambodia, Congo (9)

The whole world: Global


In [243]:
# USA data set
df_usa = df.loc[df['Country'] == 'USA']
df_usa = df_usa.reset_index().drop(columns='index').drop(columns='level_0')
df_usa

,Country,ISO 3166-1 alpha-3,Year,Total,Coal,Oil,Gas,Cement,Flaring,Other,Per Capita
0,USA,USA,1850,19.792928,19.792928,0.000000,0.000000,0.000000,0.000000,0.000000,0.839405
1,USA,USA,1851,24.633072,24.633072,0.000000,0.000000,0.000000,0.000000,0.000000,1.009333
2,USA,USA,1852,26.791168,26.791168,0.000000,0.000000,0.000000,0.000000,0.000000,1.061839
3,USA,USA,1853,30.162048,30.162048,0.000000,0.000000,0.000000,0.000000,0.000000,1.153118
4,USA,USA,1854,33.159200,33.159200,0.000000,0.000000,0.000000,0.000000,0.000000,1.233498
...,...,...,...,...,...,...,...,...,...,...,...
167,USA,USA,2017,5210.957584,1308.479533,2314.675386,1460.391796,40.323535,59.245489,27.841845,15.800777
168,USA,USA,2018,5376.657233,1256.201494,2365.283480,1618.720337,38.970745,72.786925,24.694251,16.187923
169,USA,USA,2019,5259.143836,1070.723931,2358.831137,1673.999100,40.895869,88.383147,26.310652,15.730884
170,USA,USA,2020,4715.691109,878.972885,2049.336500,1653.677852,40.687747,67.684371,25.331754,14.037218


In [244]:
# Japan data set
df_jap = df.loc[df['Country'] == 'Japan']
df_jap = df_jap.reset_index().drop(columns='index').drop(columns='level_0')
df_jap

,Country,ISO 3166-1 alpha-3,Year,Total,Coal,Oil,Gas,Cement,Flaring,Other,Per Capita
0,Japan,JPN,1850,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Japan,JPN,1851,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Japan,JPN,1852,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Japan,JPN,1853,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Japan,JPN,1854,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
167,Japan,JPN,2017,1188.358949,458.482158,448.847338,245.692546,26.428778,0.436103,8.472026,9.382092
168,Japan,JPN,2018,1143.411908,441.964043,432.510939,233.953267,26.182944,0.423414,8.377300,9.056307
169,Japan,JPN,2019,1106.015486,432.250117,415.860472,224.230136,25.328006,0.368980,7.977774,8.792438
170,Japan,JPN,2020,1042.224016,399.366949,387.857350,222.604846,24.490267,0.344529,7.560075,8.321498


In [245]:
# Germany data set
df_ger = df.loc[df['Country'] == 'Germany']
df_ger = df_ger.reset_index().drop(columns='index').drop(columns='level_0')
df_ger

,Country,ISO 3166-1 alpha-3,Year,Total,Coal,Oil,Gas,Cement,Flaring,Other,Per Capita
0,Germany,DEU,1850,15.139648,15.139648,0.000000,0.000000,0.000000,0.000000,0.000000,0.448635
1,Germany,DEU,1851,16.869056,16.869056,0.000000,0.000000,0.000000,0.000000,0.000000,0.495347
2,Germany,DEU,1852,19.173712,19.173712,0.000000,0.000000,0.000000,0.000000,0.000000,0.559163
3,Germany,DEU,1853,20.760224,20.760224,0.000000,0.000000,0.000000,0.000000,0.000000,0.603109
4,Germany,DEU,1854,24.365600,24.365600,0.000000,0.000000,0.000000,0.000000,0.000000,0.705615
...,...,...,...,...,...,...,...,...,...,...,...
167,Germany,DEU,2017,785.616471,310.763015,279.180926,171.412164,13.408148,2.266922,8.585296,9.508290
168,Germany,DEU,2018,754.408432,295.673550,265.224228,169.548548,13.227901,2.007493,8.726712,9.100585
169,Germany,DEU,2019,707.149948,240.086494,270.910415,172.478938,13.286640,2.028361,8.359100,8.504699
170,Germany,DEU,2020,639.381013,197.357678,251.383941,167.628086,13.357491,1.814935,7.838883,7.672972


In [246]:
# China data set
df_chi = df.loc[df['Country'] == 'China']
df_chi = df_chi.reset_index().drop(columns='index').drop(columns='level_0')
df_chi

,Country,ISO 3166-1 alpha-3,Year,Total,Coal,Oil,Gas,Cement,Flaring,Other,Per Capita
0,China,CHN,1850,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,China,CHN,1851,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,China,CHN,1852,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,China,CHN,1853,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,China,CHN,1854,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
167,China,CHN,2017,10011.107108,7163.317856,1430.586816,486.359360,758.185168,2.924697,169.733211,7.098687
168,China,CHN,2018,10353.876989,7316.399776,1500.473952,575.368912,786.744610,3.350856,171.538883,7.306542
169,China,CHN,2019,10740.996069,7543.157408,1559.728160,630.171360,826.876048,3.806248,177.256846,7.554165
170,China,CHN,2020,10956.213369,7605.716883,1627.725238,687.880944,858.232613,5.118808,171.538883,7.688950


In [247]:
# India data set
df_ind = df.loc[df['Country'] == 'India']
df_ind = df_ind.reset_index().drop(columns='index').drop(columns='level_0')
df_ind

,Country,ISO 3166-1 alpha-3,Year,Total,Coal,Oil,Gas,Cement,Flaring,Other,Per Capita
0,India,IND,1850,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
1,India,IND,1851,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
2,India,IND,1852,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
3,India,IND,1853,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
4,India,IND,1854,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
167,India,IND,2017,2434.867906,1560.988150,631.087430,118.922003,121.047241,2.823083,0.0,1.798018
168,India,IND,2018,2600.446463,1677.600364,655.190293,126.201229,138.964893,2.489684,0.0,1.899518
169,India,IND,2019,2626.459473,1677.653708,674.011463,128.675994,143.664137,2.454171,0.0,1.898949
170,India,IND,2020,2445.011942,1588.060207,603.602160,127.635380,122.908202,2.805994,0.0,1.750956


In [248]:
# Brazil data set
df_bra = df.loc[df['Country'] == 'Brazil']
df_bra = df_bra.reset_index().drop(columns='index').drop(columns='level_0')
df_bra

,Country,ISO 3166-1 alpha-3,Year,Total,Coal,Oil,Gas,Cement,Flaring,Other,Per Capita
0,Brazil,BRA,1850,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000,0.000000
1,Brazil,BRA,1851,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000,0.000000
2,Brazil,BRA,1852,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000,0.000000
3,Brazil,BRA,1853,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000,0.000000
4,Brazil,BRA,1854,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
167,Brazil,BRA,2017,497.412678,67.321878,316.425222,70.687790,19.080788,17.505,6.392,2.385616
168,Brazil,BRA,2018,477.102130,65.840737,300.605153,67.417072,19.342169,17.505,6.392,2.270114
169,Brazil,BRA,2019,475.104581,61.887745,302.333585,67.121321,19.864930,17.505,6.392,2.243357
170,Brazil,BRA,2020,442.306003,56.240806,281.089414,59.026096,22.052686,17.505,6.392,2.074642


In [249]:
# Senegal data set
df_sen = df.loc[df['Country'] == 'Senegal']
df_sen = df_sen.reset_index().drop(columns='index').drop(columns='level_0')
df_sen

,Country,ISO 3166-1 alpha-3,Year,Total,Coal,Oil,Gas,Cement,Flaring,Other,Per Capita
0,Senegal,SEN,1850,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
1,Senegal,SEN,1851,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
2,Senegal,SEN,1852,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
3,Senegal,SEN,1853,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
4,Senegal,SEN,1854,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
167,Senegal,SEN,2017,10.760132,1.439952,7.258384,0.036640,2.013801,0.011355,0.0,0.709875
168,Senegal,SEN,2018,11.330392,2.260688,6.950608,0.021984,2.097112,0.000000,0.0,0.727477
169,Senegal,SEN,2019,12.761457,2.087880,7.970572,0.029304,2.673702,0.000000,0.0,0.797552
170,Senegal,SEN,2020,12.743876,1.894052,8.144750,0.031372,2.673702,0.000000,0.0,0.775358


In [250]:
# Cambodia data set
df_cam = df.loc[df['Country'] == 'Cambodia']
df_cam = df_cam.reset_index().drop(columns='index').drop(columns='level_0')
df_cam

,Country,ISO 3166-1 alpha-3,Year,Total,Coal,Oil,Gas,Cement,Flaring,Other,Per Capita
0,Cambodia,KHM,1850,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000
1,Cambodia,KHM,1851,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000
2,Cambodia,KHM,1852,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000
3,Cambodia,KHM,1853,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000
4,Cambodia,KHM,1854,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
167,Cambodia,KHM,2017,12.504842,3.612704,6.591536,0.0,2.300602,0.000000,0.0,0.789911
168,Cambodia,KHM,2018,14.388181,3.839872,7.232736,0.0,3.315573,0.000000,0.0,0.897845
169,Cambodia,KHM,2019,18.024641,4.418784,8.280640,0.0,5.325217,0.000000,0.0,1.112100
170,Cambodia,KHM,2020,18.703284,4.638542,8.739525,0.0,5.325217,0.000000,0.0,1.140663


In [251]:
# Congo data set
df_con = df.loc[df['Country'] == 'Congo']
df_con = df_con.reset_index().drop(columns='index').drop(columns='level_0')
df_con

,Country,ISO 3166-1 alpha-3,Year,Total,Coal,Oil,Gas,Cement,Flaring,Other,Per Capita
0,Congo,COG,1850,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
1,Congo,COG,1851,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
2,Congo,COG,1852,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
3,Congo,COG,1853,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
4,Congo,COG,1854,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
167,Congo,COG,2017,5.702250,0.0,1.974896,1.183472,0.406868,2.137015,0.0,1.073397
168,Congo,COG,2018,6.596230,0.0,2.005842,1.343328,0.271245,2.975814,0.0,1.212306
169,Congo,COG,2019,7.341473,0.0,2.024422,1.885312,0.282870,3.148868,0.0,1.317865
170,Congo,COG,2020,7.529862,0.0,1.910896,1.972983,0.282870,3.363113,0.0,1.320525


In [252]:
# Global data set
df_glob = df.loc[df['Country'] == 'Global']
df_glob = df_glob.reset_index().drop(columns='index').drop(columns='level_0')
df_glob

,Country,ISO 3166-1 alpha-3,Year,Total,Coal,Oil,Gas,Cement,Flaring,Other,Per Capita
0,Global,WLD,1850,196.896032,196.896032,0.000000,0.000000,0.000000,0.000000,0.000000,0.155935
1,Global,WLD,1851,198.804976,198.804976,0.000000,0.000000,0.000000,0.000000,0.000000,0.156647
2,Global,WLD,1852,207.550944,207.550944,0.000000,0.000000,0.000000,0.000000,0.000000,0.162709
3,Global,WLD,1853,217.209248,217.209248,0.000000,0.000000,0.000000,0.000000,0.000000,0.169417
4,Global,WLD,1854,255.138976,255.138976,0.000000,0.000000,0.000000,0.000000,0.000000,0.197994
...,...,...,...,...,...,...,...,...,...,...,...
167,Global,WLD,2017,36096.739276,14506.973805,12242.627935,7144.928128,1507.923185,391.992176,302.294047,4.749682
168,Global,WLD,2018,36826.506600,14746.830688,12266.016285,7529.846784,1569.218392,412.115746,302.478706,4.792753
169,Global,WLD,2019,37082.558969,14725.978025,12345.653374,7647.528220,1617.506786,439.253991,306.638573,4.775633
170,Global,WLD,2020,35264.085734,14174.564010,11191.808551,7556.290283,1637.537532,407.583673,296.301685,4.497423


In [253]:
#  plot to see total emission versus year for 'global'
fig_glob = px.line(df_glob, x="Year", y="Total")
fig_glob.update_xaxes(rangeslider_visible=True)
fig_glob.show()

In [254]:
#  plot to see total emission versus year for 'Senegal'
fig_sen = px.line(df_sen, x="Year", y="Total")
fig_sen.update_xaxes(rangeslider_visible=True)
fig_sen.show()

In [255]:
#  plot to see total emission versus year for 'USA'
fig_usa = px.line(df_usa, x="Year", y="Total")
fig_usa.update_xaxes(rangeslider_visible=True)
fig_usa.show()

In [256]:
#  plot to see total emission versus year for 'Germany'
fig_ger = px.line(df_ger, x="Year", y="Total")
fig_ger.update_xaxes(rangeslider_visible=True)
fig_ger.show()

In [257]:
# USA, Japan, Germany, China, India, Brazil, Senegal, Cambodia, Congo (9)
df_usa.to_csv('Data/usa.csv', index=False)
df_jap.to_csv('Data/japan.csv', index=False)
df_ger.to_csv('Data/Germany.csv', index=False)
df_chi.to_csv('Data/China.csv', index=False)
df_ind.to_csv('Data/India.csv', index=False)
df_bra.to_csv('Data/Brazil.csv', index=False)
df_sen.to_csv('Data/Senegal.csv', index=False)
df_cam.to_csv('Data/Cambodia.csv', index=False)
df_con.to_csv('Data/Congo.csv', index=False)
df_glob.to_csv('Data/Global.csv', index=False)
df.to_csv('Data/Cleaned_whole_data.csv', index=False)
